Install dependencies

In [ ]:
! pip install labelImg
! pip install --upgrade pyqt5 lxml
! pip install opencv-python
! pip install pandas
! pip install cpython
! pip install wget
! pip install tensorflow
! pip install lxml

Import library

In [ ]:
import tensorflow as tf
import os,wget
import zipfile
import os
import shutil

Create Workspace

In [ ]:
dir_list=['addons',
          'workspace',
          'workspace/training_demo',
          'workspace/training_demo/annotations',
          'workspace/training_demo/exported-models',
          'workspace/training_demo/models',
          'workspace/training_demo/pre-trained-models',
          'workspace/training_demo/images',
          'workspace/training_demo/images/test',
          'workspace/training_demo/images/train',
          'script',
          'script/preprossing',
          ]
for dir in dir_list:
  try:
    os.mkdir(dir)
  except:
    pass

Clone Object Detection Model

In [ ]:
main_dir=os.getcwd()
os.chdir(main_dir)
os.getcwd()
! git clone https://github.com/tensorflow/models.git

Install Object Detection Api

In [ ]:
os.chdir(main_dir)
os.chdir('models/research')
os.getcwd()
! apt-get install protobuf-compiler
! protoc object_detection/protos/*.proto --python_out=.
! cp object_detection/packages/tf2/setup.py .
! python -m pip install --use-feature=2020-resolver .
os.chdir(main_dir)

Testing Object Detection Api

In [ ]:
os.chdir(main_dir)
! python models/research/object_detection/builders/model_builder_tf2_test.py
from object_detection.utils import config_util

Open labelImg and make some label for your image

In [ ]:
! labelImg

Spilt labeled img into train and test folder (workspace/training_demo/images/)

Create label_map.pbtxt and placed inside the workspace/training_demo/annotations

Create Train and test record

In [ ]:
os.chdir(main_dir)
os.chdir('script/preprossing')
wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
os.chdir(main_dir)
! python script/preprossing/generate_tfrecord.py -x workspace/training_demo/images/train -l workspace/training_demo/annotations/label_map.pbtxt -o workspace/training_demo/annotations/train.record
! python script/preprossing/generate_tfrecord.py -x workspace/training_demo/images/test -l workspace/training_demo/annotations/label_map.pbtxt -o workspace/training_demo/annotations/test.record


Download pretrained model

In [ ]:
os.chdir(main_dir)
os.chdir('workspace/training_demo/pre-trained-models')
premodel_name = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
! tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz


First configure pipeline.config and then run next cell

In [ ]:
os.chdir(main_dir)
os.chdir('workspace/training_demo')
os.mkdir('models/'+premodel_name)
! cp {os.path.join('pre-trained-models/', premodel_name, 'pipeline.config')} {os.path.join('models',premodel_name)}


Verify changes in pipeline.config file

In [ ]:
config = config_util.get_configs_from_pipeline_file('models/'+premodel_name+'/pipeline.config')
config

Copying some file to training demo (working directory)

In [ ]:
os.chdir(main_dir)
! cp models/research/object_detection/model_main_tf2.py workspace/training_demo/
! cp models/research/object_detection/exporter_main_v2.py workspace/training_demo/

Training model

In [ ]:
os.chdir('workspace/training_demo')
! python3 model_main_tf2.py --model_dir=models/{premodel_name} --pipeline_config_path=models/{premodel_name}/pipeline.config --num_train_steps=5000
# print(f'python3 model_main_tf2.py --model_dir=models/{premodel_name} --pipeline_config_path=models/{premodel_name}/pipeline.config --num_train_steps=5000')

Exporting mogel

In [ ]:
! python exporter_main_v2.py --input_type image_tensor --pipeline_config_path models/{premodel_name}/pipeline.config --trained_checkpoint_dir models/{premodel_name}/ --output_directory exported-models/my_model

If You have any doubt refer https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/index.html